In [ ]:
import cv2
import numpy as np

import nbimporter
import util
from harris_corners import *

In [ ]:
def get_keypoints(img, draw_keypoints=True):
    if len(img) > 1:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32)

    # find harris corners -> keypoints
    corners = cv2.cornerHarris(gray,2,3,0.04)
    corners = cv2.normalize(corners, corners, alpha=0, beta=255,
                       norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    threshold=85
    rows, cols = np.nonzero(corners > threshold)
    # calculate the image gradients
    Ix = util.grad_x(gray)
    Iy = util.grad_y(gray)
    O = util.grad_orientation(Ix, Iy)
    # assign the keypoints
    keypoints = []
    for i in range(len(rows)):
        r = rows[i]; c = cols[i]
        kp = cv2.KeyPoint(c, r, _size=10, _angle=np.rad2deg(O[r,c]), _octave=0)
        keypoints.append(kp)
    # draw the keypoints on the image
    if draw_keypoints:
        cv2.drawKeypoints(img, keypoints, img,
                          flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    return img, keypoints

In [ ]:
def get_matches(img1, img2):
    img1_feat, kpts1 = get_keypoints(img1, draw_keypoints=False)
    img2_feat, kpts2 = get_keypoints(img2, draw_keypoints=False)

    # create sift instance
    sift = cv2.xfeatures2d.SIFT_create()
    # get descriptors
    descriptors1 = sift.compute(img1, kpts1)[1]
    descriptors2 = sift.compute(img2, kpts2)[1]

    # get matches
    bfm = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bfm.match(descriptors1, descriptors2)
    matches = sorted(matches, key = lambda x:x.distance)

    return img1_feat, img2_feat, kpts1, kpts2, matches

In [ ]:
def draw_keypoints(img1, img2):
    _,_,kpts1, kpts2, matches = get_matches(img1, img2)
    matched_image = np.array([])
    matched_image = cv2.drawMatches(img1, kpts1, img2, kpts2, matches[:10],
                                    flags=2, outImg=matched_image)
    return matched_image